In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os
# from nltk.tokenize import sent_tokenize
# from nltk.tokenize.punkt import PunktSentenceTokenizer as pst
from transformers import BertTokenizerFast, BatchEncoding
# PST = pst()
import warnings
from tensorflow.keras.callbacks import *
import tensorflow as tf
import re
import random

In [2]:
from brat_parser import get_entities_relations_attributes_groups


In [3]:
tokenizer = BertTokenizerFast.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [4]:
def get_complete_annotations():
    train_complete_annotations_paths = ['../../n2c2/training-RiskFactors-Complete-Set1/','../../n2c2/training-RiskFactors-Complete-Set2/']
    train_complete_annotations_paths = ['../../n2c2/complete/', '../../n2c2/testing-RiskFactors-Complete']

    rows = []
    childnum = 0
    gold_rows = []
    for train_complete_annotations_path in train_complete_annotations_paths:
        for fn in os.listdir(train_complete_annotations_path):
            tree = ET.parse(train_complete_annotations_path+'/'+fn)
            root = tree.getroot()
            (TEXT,TAGS) = [child for child in root]
            notetext = TEXT.text
            for child in TAGS:
#                 print(child.tag)
                gold_row = child.attrib
                gold_row['filename'] = fn
                gold_row['note_text'] = notetext
                gold_row['tag'] = child.tag
                gold_row['observation_number'] = childnum
                gold_row['evidence_spans'] = []
                gold_row['text_cues'] = []
                row = child.attrib
                if 'start' in row.keys():
                    gold_row['evidence_spans'].append((int(row['start']),int(row['end'])))
                    gold_row['text_cues'].append(row['text'])
                for grandchild in child:
                    row = grandchild.attrib
                    row['filename'] = fn
                    row['note_text'] = notetext

#                     row['text'] = notetext
                    row['tag'] = child.tag
                    row['observation_number'] = childnum
                    rows.append(row)
                    if 'start' in row.keys():
                        gold_row['evidence_spans'].append((int(row['start']),int(row['end'])))
                        gold_row['text_cues'].append(row['text'])
                gold_rows.append(gold_row)
                childnum+=1
    return pd.DataFrame(rows), pd.DataFrame(gold_rows)


In [5]:
def get_gold_annotations():
    train_gold_annotations_paths = ['../../n2c2/training-RiskFactors-Gold-Set1/','../../n2c2/training-RiskFactors-Gold-Set2/']
    rows = []
    for train_gold_annotations_path in train_gold_annotations_paths:
        for fn in os.listdir(train_gold_annotations_path):
            tree = ET.parse(train_gold_annotations_path+'/'+fn)
            root = tree.getroot()
            (TEXT,TAGS) = [child for child in root]
            notetext = TEXT.text
            for child in TAGS:
    #             print(child.tag, child.attrib)
                row = child.attrib
                row['filename'] = fn
                row['text'] = notetext
                row['tag'] = child.tag
                rows.append(row)
    return pd.DataFrame(rows)
        


In [6]:
def get_dina_annotations():
    annotations = pd.DataFrame()

    bad=0
    for dina_annotations_path in ['../../n2c2/evens/' , '../../n2c2/odds/' ]:
        for dir in [dina_annotations_path+dir+'/' for dir in os.listdir(dina_annotations_path)]:
#             print(dir)
            RF = dir.split('/')[-1]
            if os.path.isdir(dir):
#                 print(dir)
                fns = [dir+ x for x in os.listdir(dir)]# +[dir+'/even/'+ x for x in os.listdir(dir+'/even/')]
#                 random.shuffle(fns)
                for fn in fns:
                    if '.txt' in fn:
                        with open(fn, 'r') as notefile:
                            notetext = notefile.read()
                        fn_base = fn.split('.txt')[0]
                        ann_fn = fn_base +'.ann'
                        ann_df =  pd.read_csv(ann_fn, sep='\t', names=['TA','TYPE','TEXT'])
                        entities, relations, attributes, groups = get_entities_relations_attributes_groups(ann_fn)
                        if not relations=={}:
                            return relations
                        attr_df =  pd.DataFrame([([getattr(v, x) for x in vars(v)]) for _,v in attributes.items() ], columns=['id', 'type','target', 'values'])
                        entities_df =  pd.DataFrame([([getattr(v, x) for x in vars(v)]) for _,v in entities.items() ], columns=['id', 'type','span', 'text'])
                        entities_df['note_text']= [notetext]*len(entities_df)
                        entities_df['filename']=fn.split('/')[-1]
                        entities_df['RF'] = dir.split('/')[-2]
                        if not attr_df.empty:
                            entities_df['values'] = entities_df['id'].map(attr_df.groupby('target').apply(lambda x: list(x['values'])))
                            entities_df['types'] = entities_df['id'].map(attr_df.groupby('target').apply(lambda x: list(x['type'])))
                        annotations = pd.concat([annotations,entities_df])
                        
    return annotations
                        #                     return attr_df
    #                     return entities_df


In [7]:
dina_annotations = get_dina_annotations()
# get_dina_annotations()
# os.listdir('../../n2c2/dina/' )

In [8]:
annotations_df, gold_df = get_complete_annotations()

In [9]:
gold_df.columns

Index(['id', 'time', 'type1', 'type2', 'filename', 'note_text', 'tag',
       'observation_number', 'evidence_spans', 'text_cues', 'status',
       'indicator', 'start', 'end', 'text', 'TYPE'],
      dtype='object')

In [10]:
# gold_df[gold_df['indicator'].apply(lambda x: 'bp' in str(x).lower())]

In [11]:
# pd.Series(unravel(dina_annotations[dina_annotations['type'].apply(lambda x: 'Smoker' in x)]['types'].dropna())).unique()
dina_annotations['RF'].unique()

array(['CADMention', 'HyperlipidemiaMention', 'SmokerMention', 'CADEvent',
       'ObeseMention', 'DiabetesMention', 'LDL', 'CADTestResult',
       'Cholesterol', 'BMI', 'Medication2', 'Glucose', 'A1C',
       'BloodPressure', 'CADSymptom', 'HypertensionMention',
       'Medication1'], dtype=object)

In [12]:
smoker_override = {"221-04":"current","241-01": "unknown","242-02":"past","246-02":"current", "246-03": "unknown","277-03":"never","279-01":"ever",
                            "279-02":"current","291-05":"past", "295-01": "ever","296-03":"past","298-05":"unknown","303-04":"ever",
                                                    "320-01": "unknown","320-02":"unknown","320-03": "unknown","321-03":"never","322-01": "never","399-01": "current","399-02":"never"}

In [13]:
tagdict = {'CADMention':'CAD', 'HyperlipidemiaMention':'HYPERLIPIDEMIA', 'SmokerMention':'SMOKER', 'CADEvent':'CAD',
       'ObeseMention':'OBESE', 'DiabetesMention':'DIABETES', 'LDL':'HYPERLIPIDEMIA', 'CADTestResult':'CAD',
       'Cholesterol':'HYPERLIPIDEMIA', 'BMI':'OBESE', 'Medication2':'MEDICATION', 'Glucose':'DIABETES', 'A1C':'DIABETES',
       'BloodPressure':'HYPERTENSION', 'CADSymptom':'CAD', 'HypertensionMention':'HYPERTENSION', 'Medication1':'MEDICATION'}

def transform_brat_df_row(row):
    
    if type(row['types'])==float:
        row['types']=[]
    if 'Invalid' in row['types'] or 'Negation' in row['types']:
        return np.nan
    rowdict = {x:np.nan for x in ['id', 'time', 'type1', 'type2', 'filename', 'note_text', 'tag','observation_number', 'evidence_spans', 'text_cues', 'status', 'indicator', 'start', 'end', 'text', 'TYPE']}
    rowdict['tag'] = tagdict[row['RF']]
    rowdict['filename'] = row['filename'].replace('txt','xml')
    rowdict['note_text']=row['note_text']
    if 'Time_During'in row['types']:
        rowdict['time']='during DCT'
    elif 'Time_After'in row['types']:
        rowdict['time'] = 'after DCT'
    elif 'Time_Before' in row['types']:
        rowdict['time']= 'before DCT'
    else:
        rowdict['time']=np.nan
    if 'Medication' in row['RF']:
        if row['type'].startswith('Gold_') :
            drugs=row['type'][5:].replace('_',' ').split('+')
        elif row['type'].startswith('Gold') :
            drugs=row['type'][4:].replace('_',' ').split('+')
        else:
            drugs =row['type'].replace('_',' ').split('+')
        rowdict['type1']=drugs[0]
        if len(drugs)==2:
            rowdict['type2'] =drugs[1]
        rowdict['tag']='MEDICATION'
    elif 'Smoker' in row['RF']:
        rowdict['tag']='SMOKER'
        rowdict['status'] = 'brat' 
        if row['filename'].split('.')[0] in smoker_override.keys():
            rowdict['status'] = smoker_override[row['filename'].split('.')[0]]
            print('smoker override'+row['filename'])
    rowdict['evidence_spans'] = list(row['span'])
    rowdict['text_cues'] = [row['text']]
    return rowdict

In [14]:
transform_brat_df_row(dina_annotations.iloc[3])

{'id': nan,
 'time': 'during DCT',
 'type1': nan,
 'type2': nan,
 'filename': '244-01.xml',
 'note_text': '\n\n\nRecord date: 2089-01-27\n\nPotomac Valley Hospital\n\nPreadmission Testing Area\n\nHistory & Physical Examination\n\nand Anesthesia Assessment\n\nYvonne Easton, N.P.\n\nJenna Corona, N.P.\n\nXochitl Narvaez, N.P.\n\n\n\nFrederick Q. Valladares, M.D.\n\nMedical Director\n\nJanuary 27, 2089\n\n\n\nMedical Record Number:  8751045  Francisco, Xue\n\n\n\nProcedure:  The patient is a 56-year-old Spanish-speaking female scheduled for laparoscopic cholecystectomy.  Surgery is scheduled on January 28, 2089 by Dr. Xavier Israel kenneth Xenos.\n\n\n\nChief Complaint & Present Illness:  Patient briefly evaluated on 12/30/88 as part of preoperative evaluation for a laparoscopic cholecystectomy then scheduled for 12/31/88.  Over the past year patient has had difficulty with right upper quadrant pain as well as epigastric pain.  These episodes tend to happen more frequently after she has h

In [15]:
dina_rows = dina_annotations.apply(transform_brat_df_row,axis=1)

smoker override296-03.txt
smoker override242-02.txt
smoker override322-01.txt
smoker override246-02.txt
smoker override221-04.txt
smoker override291-05.txt
smoker override295-01.txt
smoker override277-03.txt
smoker override279-01.txt
smoker override303-04.txt
smoker override303-04.txt
smoker override399-01.txt
smoker override399-01.txt
smoker override279-02.txt
smoker override279-02.txt
smoker override241-01.txt
smoker override321-03.txt
smoker override321-03.txt


In [16]:
dina_df = pd.DataFrame(list(dina_rows.dropna()))

In [17]:
set(gold_df['tag'].unique())-set(dina_df['tag'].unique())
set(dina_df['note_text'].unique())-set(gold_df['note_text'].unique())

set()

In [18]:
gold_df=pd.concat([gold_df, dina_df])

In [67]:
gold_df['tag'].unique()

array(['MEDICATION', 'SMOKER', 'FAMILY_HIST', 'PHI', 'HYPERTENSION',
       'DIABETES', 'HYPERLIPIDEMIA', 'CAD', 'OBESE'], dtype=object)

In [20]:
def get_label(row):
    if 'before' in str(row['time']):
        label='before'
    else:
        if row['tag']=='MEDICATION':
            label =  row['tag']#+row['type1']#+row['time'] 

        elif row['tag']== 'SMOKER':
            if row['status']=='never' or row['status']=='unknown':
                label='before'
            else:
                label = 'SMOKER'
        elif row['tag'] =='FAMILY_HIST':
            label =  row['tag']
        elif row['tag'] =='PHI':
            label =  'before'#row['tag']#+row['TYPE']
        elif row['tag']== 'HYPERTENSION':
            label= row['tag']#+row['time']
        elif row['tag']=='DIABETES':
            label= row['tag']
        elif row['tag']=='HYPERLIPIDEMIA':
            label= row['tag']
        elif row['tag']=='CAD':
            label= row['tag']
        elif row['tag']== 'OBESE':
            label= row['tag']

    return [label]*len(row['evidence_spans'])

In [21]:
# gold_df['encoded'] = gold_df['note_text'].map( gold_df.groupby(['note_text']).apply(lambda x: tokenizer.encode_plus(x.name, truncation=True, padding=True) ))

In [22]:
gold_df['labels'] = gold_df.apply(lambda row:get_label(row), axis=1)

In [23]:
gold_df#[gold_df['tag']=='SMOKER']

,id,time,type1,type2,filename,note_text,tag,observation_number,evidence_spans,text_cues,status,indicator,start,end,text,TYPE,labels
0,DOC0,during DCT,ACE inhibitor,,303-02.xml,"\n\n\nRecord date: 2113-03-08\n\n \nMarch 08, ...",MEDICATION,0.0,"[(863, 870), (863, 870), (863, 870)]","[Zestril, Zestril, Zestril]",NaN,NaN,NaN,NaN,NaN,NaN,"[MEDICATION, MEDICATION, MEDICATION]"
1,DOC1,during DCT,thienopyridine,,303-02.xml,"\n\n\nRecord date: 2113-03-08\n\n \nMarch 08, ...",MEDICATION,1.0,"[(1424, 1430), (1424, 1430)]","[Plavix, Plavix]",NaN,NaN,NaN,NaN,NaN,NaN,"[MEDICATION, MEDICATION]"
2,DOC2,before DCT,ACE inhibitor,,303-02.xml,"\n\n\nRecord date: 2113-03-08\n\n \nMarch 08, ...",MEDICATION,2.0,"[(863, 870), (863, 870), (863, 870)]","[Zestril, Zestril, Zestril]",NaN,NaN,NaN,NaN,NaN,NaN,"[before, before, before]"
3,DOC3,before DCT,beta blocker,,303-02.xml,"\n\n\nRecord date: 2113-03-08\n\n \nMarch 08, ...",MEDICATION,3.0,"[(908, 916), (908, 916), (908, 916)]","[Atenolol, Atenolol, Atenolol]",NaN,NaN,NaN,NaN,NaN,NaN,"[before, before, before]"
4,DOC4,after DCT,ACE inhibitor,,303-02.xml,"\n\n\nRecord date: 2113-03-08\n\n \nMarch 08, ...",MEDICATION,4.0,"[(863, 870), (863, 870), (863, 870)]","[Zestril, Zestril, Zestril]",NaN,NaN,NaN,NaN,NaN,NaN,"[MEDICATION, MEDICATION, MEDICATION]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7700,NaN,NaN,calcium channel blocker,NaN,257-02.xml,\n\n\nRecord date: 2097-03-31\n\n\n\tCARDIAC S...,MEDICATION,NaN,"[(3059, 3069)]",[amlodipine],NaN,NaN,NaN,NaN,NaN,NaN,[MEDICATION]
7701,NaN,during DCT,aspirin,NaN,257-02.xml,\n\n\nRecord date: 2097-03-31\n\n\n\tCARDIAC S...,MEDICATION,NaN,"[(7273, 7276)]",[ASA],NaN,NaN,NaN,NaN,NaN,NaN,[MEDICATION]
7702,NaN,during DCT,beta blocker,NaN,257-02.xml,\n\n\nRecord date: 2097-03-31\n\n\n\tCARDIAC S...,MEDICATION,NaN,"[(7302, 7311)]",[Lopressor],NaN,NaN,NaN,NaN,NaN,NaN,[MEDICATION]
7703,NaN,during DCT,aspirin,NaN,321-01.xml,\n\n\nRecord date: 2068-01-04\n\n ...,MEDICATION,NaN,"[(2261, 2264)]",[ASA],NaN,NaN,NaN,NaN,NaN,NaN,[MEDICATION]


In [24]:
# gold_df['encoded'].iloc[0].char_to_token(870)

In [25]:
# gold_df[gold_df.apply(lambda row: row['tag']=='OBESE',axis=1)]

In [26]:
# gold_df[gold_df['tag']=='SMOKER']

In [27]:
# gold_df[['note_text','encoded_evidence_indices', 'labels']]#.groupby(['note_text']).apply(lambda x: list(x['labels']))

In [28]:
def unravel(lst):
    return [i for j in lst for i in j]

In [29]:
def get_encoded_indices(row):
    ret = []
    for span in row['evidence_spans']:
        ret.append( [row['encoded'].char_to_token(i) for i in range(span[0],span[1])] )
    return ret
def get_all_labels(row):
    ret = ['O']*len(row['encoded']['input_ids'])
    for i,span in enumerate(row['evidence_spans']):
        ids =  [row['encoded'].char_to_token(j) for j in range(span[0],span[1])] 

        for id in ids:
            if id is not None:
                if row['labels'][i] is not 'before':
                    ret[id]='I-'+row['labels'][i]
#         for id in ids:
#             if id is not None:
#                 ret[id]='B-'+row['labels'][i]
#                 break
    if ret[0]!='O':
        ret[0] = 'B'+ret[0][1:]
    if ret[-1]!='O' and ret[-2]!=ret[-1]:
        ret[-1] = 'B'+ret[-1][1:]
    for i in range(1,len(ret)-1):
        if ret[i-1][1:]!=ret[i][1:] and ret[i]!='O':
            ret[i] = 'B'+ret[i][1:]
    return ret

In [33]:
grouped =gold_df.groupby('note_text')

In [34]:
# gold_df

In [35]:
# grouped['note_text'].apply(lambda x: x.iloc[0])
notes_df= grouped.agg({'evidence_spans':unravel, 'labels':unravel,'filename':lambda x: x.iloc[0] }).reset_index()

In [36]:
# notes_df#[notes_df['note_text'].apply(lambda x: 'LAUREN' in x)]

In [37]:
# notes_df.columns=['note_text', 'labels','evidence_spans']
notes_df['encoded'] = notes_df['note_text'].apply(lambda x: tokenizer.encode_plus(x, truncation=False, is_split_into_words=False, padding='max_length', max_length=512) )

In [38]:
notes_df['encoded'].apply(lambda x: len(x['input_ids'])).mean()

1181.1633435582821

In [39]:
#  gold_df.groupby('note_text')['note_text'].apply(lambda x:x).iloc[0]
# notes_df[notes_df['note_text'].apply(lambda x: 'Fern' in x)]

In [40]:
notes_df['token_labels'] = notes_df.apply(lambda row: get_all_labels(row), axis=1)

In [41]:
notes_df

,note_text,evidence_spans,labels,filename,encoded,token_labels
0,"\n\n\n\n\n\n\n\n\n\nHALL, NICK\n\n65219816\n\n...","[(2811, 2819), (2811, 2819), (2811, 2819), (40...","[MEDICATION, MEDICATION, MEDICATION, before, b...",292-05.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,\n\n\n\n\n\n\n OFFICE...,"[(340, 347), (340, 347), (340, 347), (1588, 16...","[MEDICATION, MEDICATION, MEDICATION, HYPERTENS...",286-04.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,\n\n\n\n\nRecord date: 2091-07-02\n\n\n\n\tCAR...,"[(555, 569), (555, 569), (555, 569), (2212, 22...","[before, before, before, before, before, HYPER...",237-01.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,\n\n\n\nRecord date: 2122-02-23\n\n\n**The sou...,"[(2033, 2043), (2033, 2043), (2033, 2043), (28...","[MEDICATION, MEDICATION, MEDICATION, MEDICATIO...",333-01.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,\n\n\nRecord Date: 2070-12-01\n\nNarrative His...,"[(1834, 1841), (1834, 1841), (1843, 1853), (18...","[before, before, before, MEDICATION, MEDICATIO...",220-03.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...,...,...,...
1299,\n\n\nRecord date: 2164-08-24\n\n\n\n\t\t\tCAR...,"[(1734, 1744), (5164, 5173), (1734, 1743), (17...","[MEDICATION, MEDICATION, MEDICATION, MEDICATIO...",227-05.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1300,\n\n\nRecord date: 2170-02-17\n\n \n\nReason f...,"[(1138, 1158), (1138, 1158), (1138, 1158), (11...","[HYPERLIPIDEMIA, HYPERLIPIDEMIA, HYPERLIPIDEMI...",200-01.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1301,\n\n\nRecord date: 2170-03-02\n\n \n\nReason f...,"[(1165, 1185), (1165, 1185), (1165, 1185), (11...","[HYPERLIPIDEMIA, HYPERLIPIDEMIA, HYPERLIPIDEMI...",200-02.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1302,\n\n\nRecord date: 2171-09-20\n\n\n\n09/20/217...,"[(1343, 1366), (1344, 1364), (1344, 1366), (13...","[before, before, before, HYPERLIPIDEMIA, HYPER...",200-03.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [42]:
# [tokenizer.decode([x]) for x in notes_df.iloc[787]['encoded']['input_ids']]
# print(notes_df.iloc[787]['note_text'])
# list(zip([tokenizer.decode([x]) for x in notes_df.iloc[0]['encoded']['input_ids']],notes_df.iloc[0]['token_labels']))
# list(zip(notes_df.iloc[0]['evidence_spans'],notes_df.iloc[0]['labels']))
# notes_df.iloc[787]['note_text']
# notes_df[notes_df['note_text'].apply(lambda x: '66 yo' in x and 'HISTORY' in x)]#.iloc[-1]['note_text']

In [43]:
# labels_list = sorted(list(set(unravel(notes_df['token_labels']))))
labels_list =['B-CAD', 'B-DIABETES', 'B-FAMILY_HIST', 'B-HYPERLIPIDEMIA', 'B-HYPERTENSION', 'B-MEDICATION', 'B-OBESE', 'B-PHI', 'B-SMOKER', 'I-CAD', 'I-DIABETES', 'I-FAMILY_HIST', 'I-HYPERLIPIDEMIA', 'I-HYPERTENSION', 'I-MEDICATION', 'I-OBESE', 'I-PHI', 'I-SMOKER', 'O']
print(labels_list)

['B-CAD', 'B-DIABETES', 'B-FAMILY_HIST', 'B-HYPERLIPIDEMIA', 'B-HYPERTENSION', 'B-MEDICATION', 'B-OBESE', 'B-PHI', 'B-SMOKER', 'I-CAD', 'I-DIABETES', 'I-FAMILY_HIST', 'I-HYPERLIPIDEMIA', 'I-HYPERTENSION', 'I-MEDICATION', 'I-OBESE', 'I-PHI', 'I-SMOKER', 'O']


In [44]:
labels_dict = {k:i for i,k in enumerate(labels_list)}

In [45]:
# from transformers import BertTokenizer, TFBertForTokenClassification
# import tensorflow as tf
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# model = TFBertForTokenClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT',from_pt=True, num_labels=len(labels_list))
# inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
# input_ids = inputs["input_ids"]
# inputs["labels"] = tf.reshape(tf.constant([1] * tf.size(input_ids).numpy()), (-1, tf.size(input_ids))) # Batch size 1
# outputs = model(inputs)
# loss = outputs.loss
# logits = outputs.logits
# model.compile()

In [46]:
# tokenizer(tokenizer.pad_token)

In [47]:
# [list(np.split(lst, range(510, lst.shape[0], 510)))
# notes_df

In [117]:
def chunk_ids(lst):
    lst=lst[1:-1]
    chunks=np.split(lst, range(510, len(lst), 510))
    ret = []
    for chunk in chunks:
        if len(chunk)<510:
            chunk=list(chunk)+[0]*(510-len(chunk))
        ret.append( np.array([101]+list(chunk)+[102]))
    return ret

def chunk_idxs(lst):
    lst=lst[1:-1]
    chunks=np.split(lst, range(510, len(lst), 510))
    ret = []
    for chunk in chunks:
        if len(chunk)<510:
            chunk=list(chunk)
        ret.append( np.array(list(chunk)))
    return ret

def chunk_labels(lst):
    lst=lst[1:-1]
    chunks=np.split(lst, range(510, len(lst), 510))
    ret = []
    for chunk in chunks:
        if len(chunk)<510:
            chunk=list(chunk)+['O']*(510-len(chunk))
        ret.append( np.array(['O']+list(chunk)+['O']))
    return ret

def chunk_ids_with_overlap(lst):
    
    if len(lst)<=512:
        return chunk_ids(lst)
    return chunk_ids(lst)+chunk_ids(lst[256:])
def chunk_labels_with_overlap(lst):
    if len(lst)<=512:
        return chunk_labels(lst)
    return chunk_labels(lst)+chunk_labels(lst[256:])

def chunk_idx_with_overlap(lst):
    if len(lst)<=512:
        return chunk_idxs(lst)
    return chunk_idxs(lst)+chunk_idxs(lst[256:])


In [118]:
# chunk_tokens(notes_df['token_labels'].iloc[0])
# chunk_ids(lst)

In [119]:
notes_df['chunked_tokens'] = notes_df['encoded'].apply(lambda x: chunk_ids_with_overlap(x['input_ids']))

In [120]:
notes_df['chunked_labels'] = notes_df['token_labels'].apply(lambda x: chunk_labels_with_overlap(x))

In [121]:
notes_df['chunked_idxs'] = notes_df['token_labels'].apply(lambda x: chunk_idx_with_overlap(list(range(len(x)))))

In [131]:
# notes_df['encoded'].iloc[0].token_to_chars(1).start

In [52]:
notes_df['patient'] = notes_df['filename'].apply(lambda x: x.split('-')[0])

In [53]:
len(notes_df['patient'].unique())

296

In [54]:
# print(list(pd.Series(notes_df['patient'].unique()).sample(30)))
val_filenames = ['259-04.xml', '259-03.xml', '357-02.xml', '185-01.xml', '156-01.xml', '337-03.xml', '223-03.xml', '361-02.xml', '128-04.xml', '351-02.xml', '397-02.xml', '121-01.xml', '397-03.xml', '149-02.xml', '329-03.xml', '174-01.xml', '360-03.xml', '128-01.xml', '145-01.xml', '368-03.xml', '282-04.xml', '176-04.xml', '326-03.xml', '148-02.xml', '103-03.xml', '157-03.xml', '304-03.xml', '395-03.xml', '322-05.xml', '333-01.xml']
# val_patients = ['366', '255', '365', '256', '129', '174', '294', '274', '123', '105', '287', '367', '185', '146', '144', '320', '333', '142', '224', '270', '297', '324', '171', '276', '298', '151', '225', '305', '332', '397']

In [55]:
# i=5
# j=-1
# print(len(notes_df['token_labels'].iloc[i]))
# pd.DataFrame(zip([tokenizer.decode([x]) for x in notes_df['chunked_tokens'].iloc[i][j]],notes_df['chunked_labels'].iloc[i][j])).tail(50)

In [70]:
notes_df

,note_text,evidence_spans,labels,filename,encoded,token_labels,chunked_tokens,chunked_labels,patient
0,"\n\n\n\n\n\n\n\n\n\nHALL, NICK\n\n65219816\n\n...","[(2811, 2819), (2811, 2819), (2811, 2819), (40...","[MEDICATION, MEDICATION, MEDICATION, before, b...",292-05.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[101, 2885, 117, 11437, 2158, 2625, 18202, 15...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",292
1,\n\n\n\n\n\n\n OFFICE...,"[(340, 347), (340, 347), (340, 347), (1588, 16...","[MEDICATION, MEDICATION, MEDICATION, HYPERTENS...",286-04.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[101, 1701, 3805, 13936, 5674, 5053, 2430, 11...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",286
2,\n\n\n\n\nRecord date: 2091-07-02\n\n\n\n\tCAR...,"[(555, 569), (555, 569), (555, 569), (2212, 22...","[before, before, before, before, before, HYPER...",237-01.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[101, 1647, 2236, 131, 21040, 1475, 118, 5004...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",237
3,\n\n\n\nRecord date: 2122-02-23\n\n\n**The sou...,"[(2033, 2043), (2033, 2043), (2033, 2043), (28...","[MEDICATION, MEDICATION, MEDICATION, MEDICATIO...",333-01.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[101, 1647, 2236, 131, 19538, 1477, 118, 5507...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",333
4,\n\n\nRecord Date: 2070-12-01\n\nNarrative His...,"[(1834, 1841), (1834, 1841), (1843, 1853), (18...","[before, before, before, MEDICATION, MEDICATIO...",220-03.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[101, 1647, 2236, 131, 21606, 1568, 118, 1367...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",220
...,...,...,...,...,...,...,...,...,...
1299,\n\n\nRecord date: 2164-08-24\n\n\n\n\t\t\tCAR...,"[(1734, 1744), (5164, 5173), (1734, 1743), (17...","[MEDICATION, MEDICATION, MEDICATION, MEDICATIO...",227-05.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[101, 1647, 2236, 131, 22148, 1527, 118, 4775...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",227
1300,\n\n\nRecord date: 2170-02-17\n\n \n\nReason f...,"[(1138, 1158), (1138, 1158), (1138, 1158), (11...","[HYPERLIPIDEMIA, HYPERLIPIDEMIA, HYPERLIPIDEMI...",200-01.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[101, 1647, 2236, 131, 22196, 1568, 118, 5507...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",200
1301,\n\n\nRecord date: 2170-03-02\n\n \n\nReason f...,"[(1165, 1185), (1165, 1185), (1165, 1185), (11...","[HYPERLIPIDEMIA, HYPERLIPIDEMIA, HYPERLIPIDEMI...",200-02.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[101, 1647, 2236, 131, 22196, 1568, 118, 5347...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",200
1302,\n\n\nRecord date: 2171-09-20\n\n\n\n09/20/217...,"[(1343, 1366), (1344, 1364), (1344, 1366), (13...","[before, before, before, HYPERLIPIDEMIA, HYPER...",200-03.xml,"[input_ids, token_type_ids, attention_mask]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[101, 1647, 2236, 131, 22196, 1475, 118, 4925...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",200


In [57]:
# notes_df['encoded'].iloc[2]
notes_df.apply(lambda row: [row['filename']*len(row['chunked_labels'])],axis=1)

0       [292-05.xml292-05.xml292-05.xml292-05.xml292-0...
1                        [286-04.xml286-04.xml286-04.xml]
2       [237-01.xml237-01.xml237-01.xml237-01.xml237-0...
3       [333-01.xml333-01.xml333-01.xml333-01.xml333-0...
4       [220-03.xml220-03.xml220-03.xml220-03.xml220-0...
                              ...                        
1299    [227-05.xml227-05.xml227-05.xml227-05.xml227-0...
1300    [200-01.xml200-01.xml200-01.xml200-01.xml200-0...
1301    [200-02.xml200-02.xml200-02.xml200-02.xml200-0...
1302           [200-03.xml200-03.xml200-03.xml200-03.xml]
1303    [200-04.xml200-04.xml200-04.xml200-04.xml200-0...
Length: 1304, dtype: object

In [122]:
train_data = pd.DataFrame(list(zip(unravel(notes_df['chunked_tokens']), unravel(notes_df['chunked_labels']),  unravel(notes_df['chunked_idxs']))) , columns=['chunked_tokens','chunked_labels', 'chunked_idxs'])

In [123]:
train_data['label_ids'] = train_data['chunked_labels'].apply(lambda x: [labels_dict[y] for y in x])

In [124]:
train_data['filename']=unravel(unravel(notes_df.apply(lambda row: [[row['filename']]*len(row['chunked_labels'])],axis=1)))

In [132]:
train_data['note_text']=unravel(unravel(notes_df.apply(lambda row: [[row['note_text']]*len(row['chunked_labels'])],axis=1)))

In [125]:
train_data['encoded']=unravel(unravel(notes_df.apply(lambda row: [[row['encoded']]*len(row['chunked_labels'])],axis=1)))

In [126]:
train_data['evidence_spans']=unravel(unravel(notes_df.apply(lambda row: [[row['evidence_spans']]*len(row['chunked_labels'])],axis=1)))

In [135]:
train_data['labels']=unravel(unravel(notes_df.apply(lambda row: [[row['labels']]*len(row['chunked_labels'])],axis=1)))

In [100]:
notes_df[notes_df['filename']=='185-01.xml']['labels'].values

array([list(['HYPERLIPIDEMIA', 'HYPERLIPIDEMIA', 'HYPERLIPIDEMIA', 'HYPERLIPIDEMIA', 'HYPERLIPIDEMIA', 'HYPERLIPIDEMIA', 'before', 'before', 'before', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'before', 'before', 'before', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'before', 'before', 'before', 'MEDICATION', 'MEDICATION', 'MEDICATION', 'before', 'before', 'before', 'before', 'before', 'before', 'HYPERTENSION', 'HYPERTENSION', 'HYPERTENSION', 'HYPERTENSION', 'HYPERTENSION', 'HYPERTENSION', 'before', 'before', 'before', 'before', 'DIABETES', 'DIABETES', 'DIABETES', 'before', 'before', 'before', 'DIABETES', 'DIABETES', 'DIABETES'])],
      dtype=object)

In [62]:
train_inputs_dict = {'input_ids':np.array([x for x in train_data[train_data['filename'].apply( lambda x: x not in val_filenames)]['chunked_tokens']])}
val_inputs_dict = {'input_ids':np.array([x for x in train_data[train_data['filename'].apply( lambda x: x in val_filenames)]['chunked_tokens']])}

In [133]:
train_data

,chunked_tokens,chunked_labels,chunked_idxs,label_ids,filename,encoded,evidence_spans,note_text
0,"[101, 2885, 117, 11437, 2158, 2625, 18202, 158...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",292-05.xml,"[input_ids, token_type_ids, attention_mask]","[(2811, 2819), (2811, 2819), (2811, 2819), (40...","\n\n\n\n\n\n\n\n\n\nHALL, NICK\n\n65219816\n\n..."
1,"[101, 8184, 1146, 117, 1117, 1250, 3750, 1144,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[511, 512, 513, 514, 515, 516, 517, 518, 519, ...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",292-05.xml,"[input_ids, token_type_ids, attention_mask]","[(2811, 2819), (2811, 2819), (2811, 2819), (40...","\n\n\n\n\n\n\n\n\n\nHALL, NICK\n\n65219816\n\n..."
2,"[101, 1144, 1793, 1105, 2604, 170, 3443, 1104,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1021, 1022, 1023, 1024, 1025, 1026, 1027, 102...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",292-05.xml,"[input_ids, token_type_ids, attention_mask]","[(2811, 2819), (2811, 2819), (2811, 2819), (40...","\n\n\n\n\n\n\n\n\n\nHALL, NICK\n\n65219816\n\n..."
3,"[101, 178, 6639, 1643, 1105, 9233, 1920, 11437...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[257, 258, 259, 260, 261, 262, 263, 264, 265, ...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",292-05.xml,"[input_ids, token_type_ids, attention_mask]","[(2811, 2819), (2811, 2819), (2811, 2819), (40...","\n\n\n\n\n\n\n\n\n\nHALL, NICK\n\n65219816\n\n..."
4,"[101, 117, 1868, 17030, 10399, 4214, 17713, 18...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[767, 768, 769, 770, 771, 772, 773, 774, 775, ...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",292-05.xml,"[input_ids, token_type_ids, attention_mask]","[(2811, 2819), (2811, 2819), (2811, 2819), (40...","\n\n\n\n\n\n\n\n\n\nHALL, NICK\n\n65219816\n\n..."
...,...,...,...,...,...,...,...,...
6450,"[101, 27466, 1306, 11509, 19756, 1394, 1406, 1...","[O, B-MEDICATION, I-MEDICATION, I-MEDICATION, ...","[1021, 1022, 1023, 1024, 1025, 1026, 1027, 102...","[18, 5, 14, 14, 14, 14, 18, 18, 18, 18, 18, 18...",200-04.xml,"[input_ids, token_type_ids, attention_mask]","[(2811, 2831), (2811, 2831), (2811, 2831), (28...",\n\n\nRecord date: 2173-02-04\n\n\nRADIATION O...
6451,"[101, 1189, 1111, 1140, 1106, 1129, 1562, 1120...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1531, 1532, 1533, 1534, 1535, 1536, 1537, 153...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",200-04.xml,"[input_ids, token_type_ids, attention_mask]","[(2811, 2831), (2811, 2831), (2811, 2831), (28...",\n\n\nRecord date: 2173-02-04\n\n\nRADIATION O...
6452,"[101, 170, 5581, 1214, 118, 1385, 12400, 1114,...","[O, O, O, O, O, O, O, O, O, B-SMOKER, I-SMOKER...","[257, 258, 259, 260, 261, 262, 263, 264, 265, ...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 8, 17, 17...",200-04.xml,"[input_ids, token_type_ids, attention_mask]","[(2811, 2831), (2811, 2831), (2811, 2831), (28...",\n\n\nRecord date: 2173-02-04\n\n\nRADIATION O...
6453,"[101, 185, 188, 5208, 1204, 22148, 1527, 6320,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[767, 768, 769, 770, 771, 772, 773, 774, 775, ...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...",200-04.xml,"[input_ids, token_type_ids, attention_mask]","[(2811, 2831), (2811, 2831), (2811, 2831), (28...",\n\n\nRecord date: 2173-02-04\n\n\nRADIATION O...


In [136]:
# train_data[train_data['filename'].apply( lambda x: x not in val_filenames)].to_csv('datasets/n2c2_train_dataset.csv')
# train_data[train_data['filename'].apply( lambda x: x in val_filenames)].to_csv('datasets/n2c2_val_dataset.csv')
train_data[train_data['filename'].apply( lambda x: x not in val_filenames)].to_pickle('datasets/n2c2_train_dataset.pk')
train_data[train_data['filename'].apply( lambda x: x in val_filenames)].to_pickle('datasets/n2c2_val_dataset.pk')

In [168]:
# pd.DataFrame.from_dict([[[row] for row in train_inputs_dict['input_ids']],list(train_data[train_data['filename'].apply( lambda x: x not in val_filenames)]['label_ids'])  ]).transpose().to_csv('datasets/n2c2_train_dataset.csv', header=['tokens','labels'])

In [169]:
# pd.DataFrame.from_dict([[[row] for row in val_inputs_dict['input_ids']],list(train_data[train_data['filename'].apply( lambda x: x not in val_filenames)]['label_ids'])  ]).transpose().to_csv('datasets/n2c2_val_dataset.csv', header=['tokens','labels'])

In [185]:
# i=767
# j=-1
# pd.DataFrame(list(zip([tokenizer.decode([x]) for x in notes_df.iloc[i]['chunked_tokens'][j]],notes_df.iloc[i]['chunked_labels'][j]))).head(30)

In [186]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    train_inputs_dict,
    list(train_data[train_data['filename'].apply( lambda x: x not in val_filenames)]['label_ids'])
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    val_inputs_dict,
    list(train_data[train_data['filename'].apply( lambda x: x in val_filenames)]['label_ids'])
))

In [187]:
optimizer =  tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08, clipnorm=1)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=optimizer)

NameError: name 'model' is not defined

In [ ]:
class MetricsCallback(Callback):
    def __init__(self):
         pass
    def on_epoch_end(self, epoch, logs):
#         try:
        self.model.save_pretrained('NER/models/bert_base_cased'+str(epoch)+'.h5')
#         except Exception as e:
#             print(concept, e)
#             warnings.warn(concept+' '+str(e))
metrics_callback = MetricsCallback()

In [ ]:
# history = model.fit(train_dataset,steps_per_epoch=1, epochs=5,validation_data=val_dataset,callbacks=[metrics_callback])

In [79]:
model = TFBertForTokenClassification.from_pretrained('NER/models/clinicalbert_batchsize8_2.h5')

Some layers from the model checkpoint at NER/models/clinicalbert_batchsize8_2.h5 were not used when initializing TFBertForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at NER/models/clinicalbert_batchsize8_2.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [80]:
outputs=model.predict(val_dataset.batch(8))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x2aaab3b58b38> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x2aaab3b58b38> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



In [81]:
# outputs

In [82]:
outputs[0].shape

(132, 512, 31)

In [83]:
predictions = tf.argmax(outputs[0], axis=2)

In [84]:
labels_list =['B-CAD', 'B-DIABETES', 'B-FAMILY_HIST', 'B-HYPERLIPIDEMIA', 'B-HYPERTENSION', 'B-MEDICATION', 'B-OBESE', 'B-PHI', 'B-SMOKER', 'I-CAD', 'I-DIABETES', 'I-FAMILY_HIST', 'I-HYPERLIPIDEMIA', 'I-HYPERTENSION', 'I-MEDICATION', 'I-OBESE', 'I-PHI', 'I-SMOKER', 'O']

# labels_list = ['I-PHI', 'I-SMOKERever', 'I-MEDICATIONdiuretic', 'I-MEDICATIONARB', 'I-MEDICATIONmetformin', 'I-CAD', 'I-SMOKERpast', 'I-SMOKERnever', 'I-MEDICATIONanti diabetes', 'I-HYPERLIPIDEMIA', 'I-MEDICATIONezetimibe', 'I-FAMILY_HIST', 'I-MEDICATIONniacin', 'I-MEDICATIONinsulin', 'I-MEDICATIONthienopyridine', 'I-MEDICATIONnitrate', 'I-MEDICATIONthiazolidinedione', 'I-MEDICATIONaspirin', 'I-MEDICATIONstatin', 'I-HYPERTENSION', 'I-MEDICATIONcalcium channel blocker', 'I-MEDICATIONACE inhibitor', 'I-MEDICATIONDPP4 inhibitors', 'I-MEDICATIONsulfonylureas', 'I-DIABETES', 'O', 'I-MEDICATIONbeta blocker', 'I-SMOKERcurrent', 'I-OBESE', 'I-SMOKERunknown', 'I-MEDICATIONfibrate']  

In [85]:
reverse_labels_dict = {i:k for i,k in enumerate(labels_list)}

In [86]:
# reverse_labels_dict

In [87]:
num_val_exs=int(predictions.numpy().shape[0]/512)

In [88]:
# val_results_df = pd.DataFrame([list(predictions.numpy().reshape([num_val_exs,512]))]).transpose()
val_results_df = pd.DataFrame([list(predictions.numpy())]).transpose()

In [89]:
train_data[train_data['filename'].apply( lambda x: x in val_filenames)]['chunked_tokens']

9       [101, 1647, 2236, 131, 19538, 1477, 118, 5507,...
10      [101, 116, 120, 8347, 1358, 191, 2875, 1181, 1...
11      [101, 1406, 120, 19538, 1477, 14516, 9238, 215...
12      [101, 120, 188, 1477, 119, 1103, 1762, 1110, 2...
13      [101, 5674, 14046, 20504, 119, 1156, 5257, 112...
                              ...                        
3811    [101, 1647, 2236, 131, 18615, 1568, 118, 4925,...
3812    [101, 9097, 120, 8710, 1106, 2584, 9964, 117, ...
3813    [101, 2047, 2052, 117, 1649, 1119, 1110, 2218,...
3814    [101, 117, 176, 1193, 19364, 3269, 117, 181, 2...
3815    [101, 3179, 1496, 1106, 2489, 119, 1119, 1674,...
Name: chunked_tokens, Length: 132, dtype: object

In [90]:
val_results_df['tokens'] = list(train_data[train_data['filename'].apply( lambda x: x in val_filenames)]['chunked_tokens'].apply(lambda x: [tokenizer.decode([y]) for y in x]) )

In [91]:
val_results_df.columns=['class_nums','tokens']

In [92]:
val_results_df['classes']=val_results_df['class_nums'].apply(lambda x: [reverse_labels_dict[y] for y in x] )

In [93]:
val_y = np.concatenate([y for x, y in val_dataset], axis=0).reshape(132,512)

In [94]:
pd.Series(list(val_y))

0      [25, 25, 25, 25, 0, 0, 0, 0, 0, 0, 25, 25, 25,...
1      [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...
2      [25, 0, 0, 0, 0, 25, 25, 25, 25, 25, 25, 25, 2...
3      [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...
4      [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...
                             ...                        
127    [25, 29, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...
128    [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...
129    [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...
130    [25, 18, 23, 23, 23, 23, 23, 21, 21, 21, 21, 2...
131    [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...
Length: 132, dtype: object

In [95]:
val_results_df['token_labels'] = pd.Series(list(val_y)).apply(lambda x: [reverse_labels_dict[y] for y in x])#list(train_data[train_data['filename'].apply( lambda x: x in val_filenames)]['chunked_labels'])

In [96]:
val_results_df['filename'] = list(train_data[train_data['filename'].apply( lambda x: x in val_filenames)]['filename'])

In [97]:
val_results_df

,class_nums,tokens,classes,token_labels,filename
0,"[25, 25, 25, 25, 0, 0, 0, 0, 0, 0, 25, 25, 25,...","[[CLS], record, date, :, 212, ##2, -, 02, -, 2...","[O, O, O, O, I-PHI, I-PHI, I-PHI, I-PHI, I-PHI...","[O, O, O, O, I-PHI, I-PHI, I-PHI, I-PHI, I-PHI...",333-01.xml
1,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[CLS], +, /, 125, ##u, v, ##it, ##d, ), 250, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",333-01.xml
2,"[25, 0, 0, 0, 0, 25, 25, 25, 25, 25, 25, 25, 2...","[[CLS], 20, /, 212, ##2, se, ##nn, ##os, ##ide...","[O, I-PHI, I-PHI, I-PHI, I-PHI, O, O, O, O, O,...","[O, I-PHI, I-PHI, I-PHI, I-PHI, O, O, O, O, O,...",333-01.xml
3,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[CLS], /, s, ##2, ., the, heart, is, regular,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, I-P...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, I-P...",333-01.xml
4,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[CLS], ##po, ##kal, ##emia, ., would, benefit...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",333-01.xml
...,...,...,...,...,...
127,"[25, 25, 25, 25, 25, 0, 25, 0, 0, 0, 0, 25, 0,...","[[CLS], record, date, :, 215, ##0, -, 09, -, 0...","[O, O, O, O, O, I-PHI, O, I-PHI, I-PHI, I-PHI,...","[O, I-SMOKERunknown, O, O, O, O, O, O, O, O, O...",185-01.xml
128,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[CLS], circulation, /, sensation, to, legs, i...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",185-01.xml
129,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[CLS], ##ri, today, ,, however, he, is, certa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",185-01.xml
130,"[25, 25, 23, 23, 23, 23, 25, 21, 21, 21, 21, 2...","[[CLS], ,, g, ##ly, ##bur, ##ide, ,, l, ##isi,...","[O, O, I-MEDICATIONsulfonylureas, I-MEDICATION...","[O, I-MEDICATIONstatin, I-MEDICATIONsulfonylur...",185-01.xml


In [117]:
from sklearn.metrics import recall_score, classification_report
recall = recall_score( unravel(list(val_y)),unravel(val_results_df['class_nums']),  average='macro')

In [118]:
recall

0.7128200925881945

In [132]:
print(classification_report(unravel(val_results_df['token_labels']),unravel(list(val_results_df['classes'])),digits=4))

                                     precision    recall  f1-score   support

                              I-CAD     0.5208    0.6897    0.5935       145
                         I-DIABETES     0.6879    0.8963    0.7784       241
                      I-FAMILY_HIST     0.3000    0.6000    0.4000        15
                   I-HYPERLIPIDEMIA     0.8837    0.8398    0.8612       181
                     I-HYPERTENSION     0.8253    0.8427    0.8339       426
          I-MEDICATIONACE inhibitor     0.8908    0.7737    0.8281       137
                    I-MEDICATIONARB     0.6842    0.5652    0.6190        23
                I-MEDICATIONaspirin     0.9167    0.6000    0.7253       110
           I-MEDICATIONbeta blocker     0.7654    0.7561    0.7607       164
I-MEDICATIONcalcium channel blocker     0.8202    0.9605    0.8848        76
               I-MEDICATIONdiuretic     1.0000    0.6757    0.8065        37
                I-MEDICATIONfibrate     0.0000    0.0000    0.0000         

In [144]:
def doc_level_stats(labels, preds):
    doc_preds = [[label in preds[i] for label in labels_list] for i in range(len(preds))]
    doc_labels = [[label in labels[i] for label in labels_list] for i in range(len(preds))]
    print(classification_report(doc_labels, doc_preds, target_names=labels_list))

In [145]:
doc_level_stats(val_results_df['token_labels'],list(val_results_df['classes']))

                                     precision    recall  f1-score   support

                              I-PHI       0.66      0.87      0.75        78
                       I-SMOKERever       0.00      0.00      0.00         0
               I-MEDICATIONdiuretic       1.00      1.00      1.00         5
                    I-MEDICATIONARB       0.60      1.00      0.75         3
              I-MEDICATIONmetformin       1.00      1.00      1.00        14
                              I-CAD       0.83      1.00      0.91        10
                       I-SMOKERpast       0.44      0.78      0.56         9
                      I-SMOKERnever       0.73      1.00      0.85        11
          I-MEDICATIONanti diabetes       0.00      0.00      0.00         0
                   I-HYPERLIPIDEMIA       0.90      0.96      0.93        27
              I-MEDICATIONezetimibe       0.00      0.00      0.00         0
                      I-FAMILY_HIST       0.67      0.67      0.67         

In [128]:
pd.DataFrame(zip(( unravel(val_results_df['token_labels']),unravel(list(val_results_df['classes'])) )))

,0
0,"[O, O, O, O, I-PHI, I-PHI, I-PHI, I-PHI, I-PHI..."
1,"[O, O, O, O, I-PHI, I-PHI, I-PHI, I-PHI, I-PHI..."


In [112]:
i = 21
pd.DataFrame(list(zip(val_results_df.iloc[i]['tokens'],val_results_df.iloc[i]['token_labels'], val_results_df.iloc[i]['classes'])), columns=['tokens', 'token_labels', 'token_preds']).head(50)

,tokens,token_labels,token_preds
0,[CLS],O,O
1,of,I-MEDICATIONbeta blocker,O
2,lab,I-MEDICATIONbeta blocker,I-MEDICATIONbeta blocker
3,##etal,I-MEDICATIONbeta blocker,I-MEDICATIONbeta blocker
4,##ol,I-MEDICATIONbeta blocker,I-MEDICATIONbeta blocker
5,p,I-MEDICATIONbeta blocker,O
6,.,I-MEDICATIONbeta blocker,O
7,o,I-MEDICATIONbeta blocker,O
8,.,O,O
9,the,I-HYPERTENSION,O


In [156]:
mimic_notes = pd.read_csv('../../../LHC_mimic/mimic3_1.4/raw_data/NOTEEVENTS.csv', nrows=100)

In [157]:
mimic_notes

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
95,20,15472,148372,2178-12-02,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2178-11-15**] ...
96,21,15472,188190,2178-12-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2178-12-5**] ...
97,22,15472,104146,2179-02-08,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2179-2-1**] D...
98,23,15472,143651,2179-03-26,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2179-3-21**] ...
